In [24]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def clean_text(text):
    return ' '.join(text.split())  # Hapus newline dan gabungkan teks menjadi satu baris

# Baca file CSV yang berisi link pertanyaan
file_path = 'HasilCrawl.csv'
df_links = pd.read_csv(file_path)

# List untuk menyimpan data detail pertanyaan
questions_data = []
for index, row in df_links.iterrows():
    link = row['Link_question']
    response = requests.get(link)

    # Periksa apakah respons berhasil
    if response.status_code == 200:
        # Parse konten HTML dengan BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')

        title = clean_text(soup.find('a', class_="question-hyperlink").text)  # Judul pertanyaan
        question_text = clean_text(soup.find('div', class_="s-prose js-post-body").text)  # Isi teks pertanyaan
        votes = clean_text(soup.find('div', class_="js-vote-count").text)  # Jumlah vote
        tags = [clean_text(tag.text) for tag in soup.find_all('a', class_="post-tag")]  # Tag pertanyaan

        # Process answers
        answers = []
        answer_areas = soup.find_all('div', class_="answer js-answer")

        if answer_areas:  # Check if there are any answers js-vote-count flex--item d-flex fd-column ai-center fc-theme-body-font fw-bold fs-subheading py4
            i = 1  # Initialize answer counter
            for answer_area in answer_areas:
                answer_text = clean_text(answer_area.find('div', class_="s-prose js-post-body").text)
                answer_votes = clean_text(answer_area.find('div', class_="js-vote-count flex--item d-flex fd-column ai-center fc-theme-body-font fw-bold fs-subheading py4").text)
                answers.append(f"[[ANSWER_{i}], [VOTES={answer_votes}]] : " + answer_text)
                i += 1


        # Simpan data ke dalam dictionary
        question_info = {
            'Number': j,
            'Title': title,
            'Question_Text': question_text,
            'Answers': answers,
            'Votes': votes,
            'Tags': ', '.join(tags),  # Gabungkan tag menjadi satu string
            'Link': link
        }
        questions_data.append(question_info)

# Simpan hasil scraping ke CSV  
output_file = 'HasilScrappingWithCSV_Request.csv'
df_questions = pd.DataFrame(questions_data)
df_questions.to_csv(output_file, index=False)
